<a href="https://colab.research.google.com/github/TejasvDua/CV/blob/main/Skin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau



In [ ]:
# Upload kaggle.json
from google.colab import files
files.upload()  # Upload kaggle.json here
# Setup Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Download Dataset
!kaggle datasets download -d shubhamgoel27/dermnet
# Unzip Dataset
!unzip -q dermnet.zip -d dermnet
# Check files
import os
print(os.listdir("/content/dermnet"))
TRAIN_PATH = "/content/dermnet/train"
TEST_PATH = "/content/dermnet/test"



Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
['train', 'test']


In [ ]:
# Create an ImageDataGenerator with data augmentation
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.3,
    rotation_range=30,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # Split training data into train and validation
)


In [ ]:
def create_model(base_model, preprocess_function):
    # Data generators
    train_generator = datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )
    test_datagen = ImageDataGenerator()
    test_generator = test_datagen.flow_from_directory(
        TEST_PATH,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )


    # Unfreeze last 30 layers for fine-tuning
    for layer in base_model.layers[-30:]:
        layer.trainable = True

    num_classes = len(train_generator.class_indices)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model, train_generator, val_generator, test_generator

In [ ]:
# ResNet50 Model
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model, resnet_train, resnet_val, resnet_test = create_model(base_model_resnet, preprocess_resnet)

# Train ResNet50
resnet_history = resnet_model.fit(
    resnet_train,
    validation_data=resnet_val,
    epochs=50,
    callbacks=[EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
               ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)]
)
resnet_model.save('resnet50_skin_disease_model.h5')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
390/390 ━━━━━━━━━━━━━━━━━━━━ 337s 701ms/step - accuracy: 0.1725 - loss: 3.3841 - val_accuracy: 0.2291 - val_loss: 2.9238 - learning_rate: 1.0000e-04
Epoch 2/50
390/390 ━━━━━━━━━━━━━━━━━━━━ 244s 626ms/step - accuracy: 0.3337 - loss: 2.3706 - val_accuracy: 0.2590 - val_loss: 2.6687 - learning_rate: 1.0000e-04
Epoch 3/50
388/390 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step - accuracy: 0.4066 - loss: 2.1015

KeyboardInterrupt: 

In [ ]:
# InceptionV3 Model
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
inception_model, inc_train, inc_val, inc_test = create_model(base_model_inception, preprocess_inception)

# Train InceptionV3
inception_history = inception_model.fit(
    inc_train,
    validation_data=inc_val,
    epochs=50,
    callbacks=[EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
               ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)]
)
inception_model.save('inceptionv3_skin_disease_model.h5')

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.
Epoch 1/50
 80/390 ━━━━━━━━━━━━━━━━━━━━ 2:34 498ms/step - accuracy: 0.1020 - loss: 3.9178

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL InternalError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/framework/errors_impl.py(462): __init__
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/execute.py(53): quick_execute
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/context.py(1683): call_function
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(251): call_flat
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(216): call_preflattened
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/concrete_function.py(1322): _call_flat
  /usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/tracing_compilation.py(139): call_function
  /usr/local/lib/pyt

In [ ]:
# Evaluate models
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(resnet_test)
inception_test_loss, inception_test_acc = inception_model.evaluate(inc_test)
print(f'ResNet50 Test Accuracy: {resnet_test_acc:.4f}, Test Loss: {resnet_test_loss:.4f}')
print(f'InceptionV3 Test Accuracy: {inception_test_acc:.4f}, Test Loss: {inception_test_loss:.4f}')

126/126 ━━━━━━━━━━━━━━━━━━━━ 19s 152ms/step - accuracy: 0.4372 - loss: 2.0171
126/126 ━━━━━━━━━━━━━━━━━━━━ 27s 164ms/step - accuracy: 0.1323 - loss: 3.4327
ResNet50 Test Accuracy: 0.4113, Test Loss: 2.1040
InceptionV3 Test Accuracy: 0.1312, Test Loss: 3.3518
